In [2]:
import numpy as np

# Membuat graf sederhana
edge_index = np.array([[0, 1, 1, 2],
                       [1, 0, 2, 1]])  # Edge connections
x = np.array([[1], [2], [3]], dtype=float)  # Node features
y = np.array([0, 1, 0])  # Node labels

# Fungsi untuk menghitung adjacency matrix
def compute_adjacency_matrix(edge_index, num_nodes):
    adj = np.zeros((num_nodes, num_nodes))
    for i in range(edge_index.shape[1]):
        src, dst = edge_index[:, i]
        adj[src, dst] = 1
    return adj

# Fungsi untuk menormalkan adjacency matrix
def normalize_adjacency_matrix(adj):
    degree = np.sum(adj, axis=1)
    degree_inv_sqrt = np.diag(1.0 / np.sqrt(degree + 1e-8))
    return degree_inv_sqrt @ adj @ degree_inv_sqrt

# Graph Convolution Layer
def graph_convolution(x, adj, weight):
    return adj @ x @ weight

# Inisialisasi parameter
num_nodes = x.shape[0]
input_dim = x.shape[1]
hidden_dim = 16
output_dim = 2

W1 = np.random.randn(input_dim, hidden_dim) * 0.01
W2 = np.random.randn(hidden_dim, output_dim) * 0.01

# Membuat adjacency matrix dan menormalkannya
adj = compute_adjacency_matrix(edge_index, num_nodes)
adj_normalized = normalize_adjacency_matrix(adj)

# Aktivasi ReLU
def relu(x):
    return np.maximum(0, x)

# Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Training loop
learning_rate = 0.01
for epoch in range(100):
    # Forward pass
    h1 = relu(graph_convolution(x, adj_normalized, W1))
    out = softmax(graph_convolution(h1, adj_normalized, W2))
    
    # Loss (Cross-Entropy)
    loss = -np.mean(np.log(out[np.arange(len(y)), y]))
    
    # Backward pass (Gradient Descent)
    grad_out = out
    grad_out[np.arange(len(y)), y] -= 1
    grad_out /= len(y)
    
    grad_W2 = h1.T @ (adj_normalized @ grad_out)
    grad_h1 = (adj_normalized @ grad_out) @ W2.T
    grad_h1[h1 <= 0] = 0  # ReLU backward
    
    grad_W1 = x.T @ (adj_normalized @ grad_h1)
    
    # Update weights
    W1 -= learning_rate * grad_W1
    W2 -= learning_rate * grad_W2
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

# Evaluasi model
h1 = relu(graph_convolution(x, adj_normalized, W1))
out = softmax(graph_convolution(h1, adj_normalized, W2))
pred = np.argmax(out, axis=1)
accuracy = np.mean(pred == y)
print(f'Accuracy: {accuracy:.4f}')

Epoch 0, Loss: 0.6931
Epoch 10, Loss: 0.6931
Epoch 20, Loss: 0.6930
Epoch 30, Loss: 0.6930
Epoch 40, Loss: 0.6930
Epoch 50, Loss: 0.6929
Epoch 60, Loss: 0.6929
Epoch 70, Loss: 0.6928
Epoch 80, Loss: 0.6928
Epoch 90, Loss: 0.6927
Accuracy: 0.6667
